### Capstone Project -Week 4

### Data Sources:
##### The following data files are used for the analysis
##### 1. <a href="https://www.mapsofworld.com/usa/zipcodes/california/los-angeles/" target="_blank">Los Angeles County City List</a> - City and Zipcode Mapping for the cities in Los Angeles County.
##### 2. <a href="https://data.lacity.org/Community-Economic-Development/2010-Census-Populations-by-Zip-Code/nxs9-385f" target = "_blank"> Los Angeles County Population and Household File </a> - Total Population, Total Male, Total Femail, Total Household, Avg. Houshold for the cities of Los Angeles County.
##### 3. <a href="https://en.wikipedia.org/wiki/Demographics_of_Los_Angeles_County" target="_blank"> Los Angeles County Household Income </a> - Medium Household Income and Medium Family Income for the cityes of Los Angeles County
##### 4. Neighborhood Venues from Foursquare based on coordinate of cities.  

In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans 

from geopy.geocoders import Nominatim
import requests

import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

##### Get the source data for City and Zipcode Mapping for the cities in Los Angeles County.

In [2]:
url="https://www.mapsofworld.com/usa/zipcodes/california/los-angeles/"
tbls=pd.read_html(url)
print(len(tbls))

10


In [3]:
city_zip_map=tbls[2]
city_zip_map.columns=['State','Borough','City','Zipcode']
city_zip_map.drop([0],inplace=True)
city_zip_map["Address"]=city_zip_map["City"]+","+city_zip_map["Borough"]+","+city_zip_map["State"]
city_zip_map['Zipcode']=pd.to_numeric(city_zip_map['Zipcode'])
city_zip_map.head()

,State,Borough,City,Zipcode,Address
1,California,Los Angeles,Acton,93510,"Acton,Los Angeles,California"
2,California,Los Angeles,Agoura Hills,91301,"Agoura Hills,Los Angeles,California"
3,California,Los Angeles,Alhambra,91801,"Alhambra,Los Angeles,California"
4,California,Los Angeles,Altadena,91001,"Altadena,Los Angeles,California"
5,California,Los Angeles,Arcadia,91006,"Arcadia,Los Angeles,California"


#### Get Latitue and Longitude for each city

In [4]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_los_angeles_agent")
city_zip_map["Loc_Coord"]=city_zip_map['Address'].apply(geolocator.geocode).apply(lambda Address:tuple(Address.point) if Address else None)
city_zip_map[["lat","log","altitude"]]=pd.DataFrame(city_zip_map["Loc_Coord"].tolist(),index=city_zip_map.index)
city_zip_map.drop(['Loc_Coord','altitude'],axis=1,inplace=True)
city_zip_map

,State,Borough,City,Zipcode,Address,lat,log
1,California,Los Angeles,Acton,93510,"Acton,Los Angeles,California",34.480741,-118.186838
2,California,Los Angeles,Agoura Hills,91301,"Agoura Hills,Los Angeles,California",34.147910,-118.765704
3,California,Los Angeles,Alhambra,91801,"Alhambra,Los Angeles,California",34.093042,-118.127060
4,California,Los Angeles,Altadena,91001,"Altadena,Los Angeles,California",34.186316,-118.135233
5,California,Los Angeles,Arcadia,91006,"Arcadia,Los Angeles,California",34.053691,-118.242766
6,California,Los Angeles,Artesia,90701,"Artesia,Los Angeles,California",33.869020,-118.079620
7,California,Los Angeles,Avalon,90704,"Avalon,Los Angeles,California",33.342210,-118.327261
8,California,Los Angeles,Azusa,91702,"Azusa,Los Angeles,California",34.133875,-117.905605
9,California,Los Angeles,Baldwin Park,91706,"Baldwin Park,Los Angeles,California",34.085474,-117.961176
10,California,Los Angeles,Bell,90202,"Bell,Los Angeles,California",33.974781,-118.186636


####  Get City Population, Median Age, Total Households and Average Household size

In [5]:
population=pd.read_csv("2010_Census_Populations_by_Zip_Code.csv")
population['Zip Code']=pd.to_numeric(population['Zip Code'])
population.head()

,Zip Code,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size
0,91371,1,73.5,0,1,1,1.00
1,90001,57110,26.6,28468,28642,12971,4.40
2,90002,51223,25.5,24876,26347,11731,4.36
3,90003,66266,26.3,32631,33635,15642,4.22
4,90004,62180,34.8,31302,30878,22547,2.73


#### Merge City Mapping file and Population file together based on Zip code.

In [6]:
population_household=pd.merge(left=population,right=city_zip_map,left_on=population['Zip Code'],right_on=city_zip_map['Zipcode'])
population_household.head()

,key_0,Zip Code,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size,State,Borough,City,Zipcode,Address,lat,log
0,90001,90001,57110,26.6,28468,28642,12971,4.40,California,Los Angeles,Los Angeles,90001,"Los Angeles,Los Angeles,California",34.053691,-118.242766
1,90069,90069,20483,41.5,12153,8330,13364,1.53,California,Los Angeles,West Hollywood,90069,"West Hollywood,Los Angeles,California",34.092301,-118.369289
2,90090,90090,0,0.0,0,0,0,0.00,California,Los Angeles,Dodgertown,90090,"Dodgertown,Los Angeles,California",NaN,NaN
3,90094,90094,5464,33.7,2559,2905,2949,1.85,California,Los Angeles,Playa Vista,90094,"Playa Vista,Los Angeles,California",33.976010,-118.418165
4,90201,90201,101279,27.8,50658,50621,24104,4.16,California,Los Angeles,Bell Gardens,90201,"Bell Gardens,Los Angeles,California",33.969456,-118.150395


In [7]:
population_household.shape

(121, 15)

#### Data cleanse

In [8]:
population_household.groupby("City").sum().reset_index()
population_household.dropna(subset=['lat','log'],inplace=True)
population_household.head()

,key_0,Zip Code,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size,State,Borough,City,Zipcode,Address,lat,log
0,90001,90001,57110,26.6,28468,28642,12971,4.40,California,Los Angeles,Los Angeles,90001,"Los Angeles,Los Angeles,California",34.053691,-118.242766
1,90069,90069,20483,41.5,12153,8330,13364,1.53,California,Los Angeles,West Hollywood,90069,"West Hollywood,Los Angeles,California",34.092301,-118.369289
3,90094,90094,5464,33.7,2559,2905,2949,1.85,California,Los Angeles,Playa Vista,90094,"Playa Vista,Los Angeles,California",33.976010,-118.418165
4,90201,90201,101279,27.8,50658,50621,24104,4.16,California,Los Angeles,Bell Gardens,90201,"Bell Gardens,Los Angeles,California",33.969456,-118.150395
5,90220,90220,49328,29.8,23773,25555,12741,3.85,California,Los Angeles,Compton,90220,"Compton,Los Angeles,California",33.894927,-118.226624


In [9]:
population_household.shape

(119, 15)

#### Get City Household income file 

In [10]:
url2="https://en.wikipedia.org/wiki/Demographics_of_Los_Angeles_County"
tbl2=pd.read_html(url2)
Population_Income=tbl2[2]
Population_Income.columns=['Place', 'Type','Population','Per capita income','Median household income','Median family income']
Population_Income.head()

,Place,Type,Population,Per capita income,Median household income,Median family income
0,Acton,CDP,7138,"$33,279","$87,896","$94,673"
1,Agoura Hills,City,20353,"$53,193","$108,820","$129,648"
2,Agua Dulce,CDP,2991,"$39,064","$97,000","$105,833"
3,Alhambra,City,83301,"$24,849","$52,717","$56,062"
4,Alondra Park,CDP,8358,"$23,742","$58,906","$57,784"


In [11]:
Population_Income.shape

(141, 6)

#### Merge the income file with the main file

In [12]:
pop_house_income=pd.merge(left=population_household,right=Population_Income,left_on="City",right_on="Place")
pop_house_income.head()

,key_0,Zip Code,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size,State,Borough,...,Zipcode,Address,lat,log,Place,Type,Population,Per capita income,Median household income,Median family income
0,90001,90001,57110,26.6,28468,28642,12971,4.40,California,Los Angeles,...,90001,"Los Angeles,Los Angeles,California",34.053691,-118.242766,Los Angeles,City,3782544,"$28,222","$50,028","$54,243"
1,90069,90069,20483,41.5,12153,8330,13364,1.53,California,Los Angeles,...,90069,"West Hollywood,Los Angeles,California",34.092301,-118.369289,West Hollywood,City,34564,"$53,227","$52,303","$63,725"
2,90201,90201,101279,27.8,50658,50621,24104,4.16,California,Los Angeles,...,90201,"Bell Gardens,Los Angeles,California",33.969456,-118.150395,Bell Gardens,City,42294,"$12,026","$38,971","$39,190"
3,90220,90220,49328,29.8,23773,25555,12741,3.85,California,Los Angeles,...,90220,"Compton,Los Angeles,California",33.894927,-118.226624,Compton,City,96102,"$13,595","$43,311","$44,835"
4,90230,90230,31766,39.1,14932,16834,12883,2.45,California,Los Angeles,...,90230,"Culver City,Los Angeles,California",34.021122,-118.396466,Culver City,City,38899,"$42,832","$75,596","$96,809"


In [13]:
pop_house_income.shape

(76, 21)

#### Data Cleanse to get the final data file for further analysis

In [14]:
pop_house_income_final=pop_house_income.drop(["key_0","Zip Code","Zipcode","Place","Type","Total Population","State","Borough"],axis=1)
pop_house_income_final.head()

,Median Age,Total Males,Total Females,Total Households,Average Household Size,City,Address,lat,log,Population,Per capita income,Median household income,Median family income
0,26.6,28468,28642,12971,4.40,Los Angeles,"Los Angeles,Los Angeles,California",34.053691,-118.242766,3782544,"$28,222","$50,028","$54,243"
1,41.5,12153,8330,13364,1.53,West Hollywood,"West Hollywood,Los Angeles,California",34.092301,-118.369289,34564,"$53,227","$52,303","$63,725"
2,27.8,50658,50621,24104,4.16,Bell Gardens,"Bell Gardens,Los Angeles,California",33.969456,-118.150395,42294,"$12,026","$38,971","$39,190"
3,29.8,23773,25555,12741,3.85,Compton,"Compton,Los Angeles,California",33.894927,-118.226624,96102,"$13,595","$43,311","$44,835"
4,39.1,14932,16834,12883,2.45,Culver City,"Culver City,Los Angeles,California",34.021122,-118.396466,38899,"$42,832","$75,596","$96,809"


In [15]:
pop_house_income_final=pop_house_income_final[['City','Address','lat','log','Population','Median Age','Total Males','Total Females','Total Households','Average Household Size','Per capita income','Median household income','Median family income']]
pop_house_income_final                          

,City,Address,lat,log,Population,Median Age,Total Males,Total Females,Total Households,Average Household Size,Per capita income,Median household income,Median family income
0,Los Angeles,"Los Angeles,Los Angeles,California",34.053691,-118.242766,3782544,26.6,28468,28642,12971,4.40,"$28,222","$50,028","$54,243"
1,West Hollywood,"West Hollywood,Los Angeles,California",34.092301,-118.369289,34564,41.5,12153,8330,13364,1.53,"$53,227","$52,303","$63,725"
2,Bell Gardens,"Bell Gardens,Los Angeles,California",33.969456,-118.150395,42294,27.8,50658,50621,24104,4.16,"$12,026","$38,971","$39,190"
3,Compton,"Compton,Los Angeles,California",33.894927,-118.226624,96102,29.8,23773,25555,12741,3.85,"$13,595","$43,311","$44,835"
4,Culver City,"Culver City,Los Angeles,California",34.021122,-118.396466,38899,39.1,14932,16834,12883,2.45,"$42,832","$75,596","$96,809"
5,El Segundo,"El Segundo,Los Angeles,California",33.917028,-118.415634,16597,39.2,8304,8350,7085,2.34,"$46,185","$88,486","$103,890"
6,Gardena,"Gardena,Los Angeles,California",33.896359,-118.305304,58743,35.5,23217,24270,15830,2.96,"$22,969","$46,961","$55,227"
7,Hawthorne,"Hawthorne,Los Angeles,California",33.918859,-118.348326,84293,31.9,45113,48080,31087,2.98,"$19,862","$45,622","$50,490"
8,Hermosa Beach,"Hermosa Beach,Los Angeles,California",33.864280,-118.395910,19422,37.0,10273,9233,9550,2.04,"$69,857","$102,289","$156,506"
9,Huntington Park,"Huntington Park,Los Angeles,California",33.982704,-118.212034,58465,29.1,37525,37541,18419,4.06,"$12,461","$36,788","$37,573"


In [16]:
pop_house_income_final.shape

(76, 13)